In [168]:
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

In [169]:
password = 'Phorulz1'

In [170]:
conn = pymysql.connect(
    host='127.0.0.1',
    port = 3306,
    user = 'root',
    password = password,
    database = 'mongo_dbs'
    
)

In [148]:
cnx = create_engine("mysql+pymysql://root:"+ password+"@localhost:3306/mongo_dbs")

In [149]:
import pandas as pd
courses = pd.read_csv('courses.csv', encoding = "ISO-8859-1", engine='python') 
fam_members = pd.read_csv('familyMembers.csv', encoding = "ISO-8859-1", engine='python') 
group_members = pd.read_csv('groupMembers.csv', encoding = "ISO-8859-1", engine='python') 
pets = pd.read_csv('pets.csv', encoding = "ISO-8859-1", engine='python')

In [150]:
group_members

,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,sibling3,...,course1,course2,course3,course4,course5,course6,course7,major,favIceCream,inState
0,1,Anna Grace Calhoun,2001/06/11,"Boston, MA","Charlottesville, VA",19,3,1.0,2.0,3.0,...,1,3,5.0,8.0,10.0,14.0,15.0,"Computer Science, Public Policy and Leadership",Chocolate,1
1,2,Daniel Tohti,2001/12/28,"Gainesville, FL","Fairfax, VA",19,2,6.0,7.0,NaN,...,0,2,12.0,4.0,6.0,7.0,9.0,Aerospace Engineering,Mint Chocolate Chip,1
2,3,Dare Nguyen,1996/09/03,"Virginia Beach, VA","Virginia Beach, VA",24,3,13.0,14.0,17.0,...,11,16,13.0,17.0,NaN,NaN,NaN,Psychology,Cookies and Cream,1
3,4,David Siamon,2014/05/30,"Reston, VA","Fairfax, VA",7,1,10.0,NaN,NaN,...,18,19,20.0,21.0,22.0,NaN,NaN,"Mathematics, Computer Science",Melted,1
4,5,Danny Devito,1944/11/17,"Neptune Township, NJ","Neptune Township, NJ",76,0,NaN,NaN,NaN,...,14,23,NaN,NaN,NaN,NaN,NaN,Acting,Chocolate,0


In [151]:
fam_members = fam_members.rename(columns = {"id ":"id"})
pets = pets.rename(columns = {"id ":"id"})

In [152]:
fam_members.columns

Index(['id', 'relation', 'name', 'age', 'gender', 'birthday', 'favIceCream',
       'petCount', 'pet1Name', 'pet2Name', 'enrolledAtUVA', 'courses',
       'groupMemberRelative'],
      dtype='object')

In [153]:
group_members.to_sql(name='groupMembers', con=cnx, if_exists = 'append', index=True)

In [154]:
fam_members.to_sql(name='familyMembers', con=cnx, if_exists = 'append', index=False)

In [155]:
pets.to_sql(name = 'pets', con=cnx, if_exists = 'append', index = False)

In [156]:
courses.to_sql(name = 'courses', con=cnx, if_exists = 'append', index=False)

In [157]:
pd.read_sql("SELECT * from groupMembers JOIN pets on groupMembers.id=pets.ownerGroup", con = cnx)


,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,favIceCream,inState,id,name,species,breed,gender,age,ownerGroup,color
0,0,1,Anna Grace Calhoun,2001/06/11,"Boston, MA","Charlottesville, VA",19,3,1,2.0,...,Chocolate,1,1,King Louie,Dog,Unknown,M,5.00,1,white
1,2,3,Dare Nguyen,1996/09/03,"Virginia Beach, VA","Virginia Beach, VA",24,3,13,14.0,...,Cookies and Cream,1,2,Taro,Dog,Border Collie mix,M,0.58,3,Brown
2,3,4,David Siamon,2014/05/30,"Reston, VA","Fairfax, VA",7,1,10,NaN,...,Melted,1,3,Blacky,Cat,None,M,13.00,4,Black
3,3,4,David Siamon,2014/05/30,"Reston, VA","Fairfax, VA",7,1,10,NaN,...,Melted,1,4,Jacky,Cat,None,M,13.00,4,Brown
4,1,2,Daniel Tohti,2001/12/28,"Gainesville, FL","Fairfax, VA",19,2,6,7.0,...,Mint Chocolate Chip,1,5,Leo,Dog,German Shepherd,M,2.80,2,Black


In [206]:
#pd.read_sql("SELECT groupMembers.name, courses.`class title` from groupMembers JOIN courses on groupMembers.course1=courses.id", con = cnx)
group_members.columns

Index(['id', 'name', 'birthday', 'birthCity', 'hometown', 'age',
       'siblingCount', 'sibling1', 'sibling2', 'sibling3', 'petCount', 'pet1',
       'pet2', 'course1', 'course2', 'course3', 'course4', 'course5',
       'course6', 'course7', 'major', 'favIceCream', 'inState'],
      dtype='object')

In [216]:
# Who's taking the same course and likes Chocolate Ice Cream -- maybe they can get ice cream after class?
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course1=courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)



,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1.0,2.0,...,1,CS,3250,Computer Science,Software Testing,Upsorn Praphamontripong,"Tu, Th",Lecture,60,An introduction to testing for assuring softwa...
1,3,4,David Siamon,2014-05-30,"Reston, VA","Fairfax, VA",7,1,10.0,NaN,...,18,CS,4750,Computer Science,Database Systems,Upsorn Praphamontripong,"M, W, F",Lecture,125,Introduces the fundamental concepts for design...
2,4,5,Danny Devito,1944-11-17,"Neptune Township, NJ","Neptune Township, NJ",76,0,NaN,NaN,...,14,GNUR,5390,Graduate Nursing,Intro to U.S. Healthcare,Rick Mayes,W,Lecture,70,This course is designed to provide an overview...


In [217]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course2=courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)


,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1.0,2.0,...,3,LPPP,3200,Leadership and Public Policy,Introduction to Public Policy,None,M,Discussion,20,This course will introduce students to both th...
1,4,5,Danny Devito,1944-11-17,"Neptune Township, NJ","Neptune Township, NJ",76,0,NaN,NaN,...,23,DVTO,4310,Danny Devito,Danny Devito,Danny Devito,"M, Tu, F",Lecture,7,Danny


In [218]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course3=courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)


,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1,2,...,5,LPPP,3200,Leadership and Public Policy,Introduction to Public Policy,Peter Johannessen,"M, W",Lecture,45,This course will introduce students to both th...
1,1,2,Daniel Tohti,2001-12-28,"Gainesville, FL","Fairfax, VA",19,2,6,7,...,12,CS,2150,Computer Science,Program & Data Representation,Aaron Bloomfield,"M, Tu, W, F",Lecture,485,Introduces programs and data representation at...


In [219]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course4=courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)

,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1,2.0,...,8,CS,4750,Computer Science,Database Systems,Nada Basit,"M, W, F",Lecture,95,Introduces the fundamental concepts for design...
1,3,4,David Siamon,2014-05-30,"Reston, VA","Fairfax, VA",7,1,10,NaN,...,21,CS,4102,Computer Science,Algorithms,Mark Floryan,"Tu, Th",Lecture,125,Introduces the analysis of algorithms and the ...


In [220]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course5=courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)

,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1,2,...,10,LPPL,3210,Leadership and Public Policy,Intro to Civic Leadership,Maura Austin,M,Lecture,35,"Drawing on social science research, this cours..."


In [221]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course6=courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)

,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1,2,...,14,GNUR,5390,Graduate Nursing,Intro to U.S. Healthcare,Rick Mayes,W,Lecture,70,This course is designed to provide an overview...


In [226]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course7=courses.id AND groupMembers.course3 = courses.id WHERE courses.`course mnemonic`='CS' OR courses.`course number` = '2150' OR groupMembers.favIceCream = 'Chocolate'",con=cnx)

,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description


In [246]:
pd.read_sql("SELECT * from groupMembers JOIN courses on groupMembers.course7 = courses.id", con=cnx)




,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,1,Anna Grace Calhoun,2001-06-11,"Boston, MA","Charlottesville, VA",19,3,1,2,...,15,LPPP,3001,Leadership and Public Policy,Public Policy Writing Lab,Staff,W,Lab,15,Develops professional writing skills.
1,1,2,Daniel Tohti,2001-12-28,"Gainesville, FL","Fairfax, VA",19,2,6,7,...,9,STS,2050,Engineering & Society,Sustainable Energy Systems,James Groves,Th,Lecture,375,This course investigates a major source of hum...


In [ ]:
# Anna Grace Calhoun and Danny are both taking GNUR 5390 and both like Chocolate ice cream.

In [312]:
# What is the average class capacity for courses people from Fairfax are taking?
pd.read_sql("SELECT groupMembers.name, groupMembers.hometown, AVG(courses.`class capacity`) FROM (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course1 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t1 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course2 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t2 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course3 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t3 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course4 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t4 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course5 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t5 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course6 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t6 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course7 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t7 GROUP BY groupMembers.name, groupMembers.hometown", con=cnx)


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'AS t2 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class cap' at line 1")
[SQL: SELECT groupMembers.name, groupMembers.hometown, AVG(courses.`class capacity`) FROM (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course1 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t1 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course2 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t2 UNION (SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course3 = courses.id WHERE groupMembers.hometown LIKE 'Fair%') AS t3 UNION (SELECT groupMembe

In [392]:
pd.read_sql("SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course1 = courses.id WHERE groupMembers.hometown LIKE 'Fair%' UNION SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course2 = courses.id WHERE groupMembers.hometown LIKE 'Fair%' UNION SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course3 = courses.id WHERE groupMembers.hometown LIKE 'Fair%' UNION SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course4 = courses.id WHERE groupMembers.hometown LIKE 'Fair%' UNION SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course5 = courses.id WHERE groupMembers.hometown LIKE 'Fair%' UNION SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course6 = courses.id WHERE groupMembers.hometown LIKE 'Fair%' UNION SELECT groupMembers.name, groupMembers.hometown, courses.`class capacity` FROM groupMembers LEFT JOIN courses on groupMembers.course7 = courses.id WHERE groupMembers.hometown LIKE 'Fair%'", con=cnx)


,name,hometown,class capacity
0,Daniel Tohti,"Fairfax, VA",300.0
1,David Siamon,"Fairfax, VA",125.0
2,Daniel Tohti,"Fairfax, VA",75.0
3,David Siamon,"Fairfax, VA",27.0
4,Daniel Tohti,"Fairfax, VA",485.0
5,David Siamon,"Fairfax, VA",320.0
6,Daniel Tohti,"Fairfax, VA",25.0
7,Daniel Tohti,"Fairfax, VA",210.0
8,David Siamon,"Fairfax, VA",25.0
9,Daniel Tohti,"Fairfax, VA",170.0


In [389]:
pd.read_sql("SELECT  FROM groupMembers INNER JOIN courses AS c1 ON groupMembers.course1 = c1.id INNER JOIN courses AS c2 ON groupMembers.course2 = c2.id WHERE groupMembers.hometown LIKE 'Fair%'", con = cnx)


,index,id,name,birthday,birthCity,hometown,age,siblingCount,sibling1,sibling2,...,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,1,2,Daniel Tohti,2001-12-28,"Gainesville, FL","Fairfax, VA",19,2,6,7.0,...,2,APMA,3501,Applied Mathematics,Spec Topics in APMA,Gianluca Guadagni,M,Lecture,75,Applies mathematical techniques to special pro...
1,3,4,David Siamon,2014-05-30,"Reston, VA","Fairfax, VA",7,1,10,NaN,...,19,HIEU,3452,European History,Jewish Culture & Hist East Eur,James Loeffler,"M, W",Lecture,27,This course is a comprehensive examination of ...


In [354]:
# Which day of the week is more popular for lectures? Labs?
pd.read_sql("SELECT DISTINCT(courses.`meeting day(s)`), COUNT(type) FROM courses WHERE type LIKE 'Le%' GROUP BY courses.`meeting day(s)`", con=cnx)
# Tuesday and Thursday are the most popular for lectures

,meeting day(s),COUNT(type)
0,"M,W,F",1
1,"Tu, Th",8
2,M,2
3,"M, W",2
4,"M, W, F",2
5,Th,1
6,"M, Tu, W, F",1
7,W,1
8,"M, T, W, Th, F",1
9,"M, Tu, F",1


In [351]:
pd.read_sql("SELECT DISTINCT(courses.`meeting day(s)`), COUNT(type) FROM courses WHERE type LIKE 'La%' GROUP BY courses.`meeting day(s)`", con=cnx)
# Monday and Wednesday are a tie for labs

,meeting day(s),COUNT(type)
0,M,1
1,W,1


In [393]:
pets.columns

Index(['id', 'name', 'species', 'breed', 'gender', 'age', 'ownerGroup',
       'color'],
      dtype='object')

In [403]:
# What is the average age of dog owners? Cat owners?
pd.read_sql("SELECT AVG(groupMembers.age), pets.species FROM groupMembers JOIN pets ON groupMembers.id = pets.ownerGroup WHERE pets.species = 'Dog' GROUP BY  pets.species", con = cnx)
# Average age is 20.6667 for dog owners

,AVG(groupMembers.age),species
0,20.6667,Dog


In [404]:
pd.read_sql("SELECT AVG(groupMembers.age), pets.species FROM groupMembers JOIN pets ON groupMembers.id = pets.ownerGroup WHERE pets.species = 'Cat' GROUP BY  pets.species", con = cnx)
# Young people love cats-- cat owners' avg age is 7

,AVG(groupMembers.age),species
0,7.0,Cat


In [424]:
# What are the average ages of each family? 
# Calhoun family
pd.read_sql("SELECT groupMembers.name, groupMembers.age, AVG(familyMembers.age) FROM groupMembers JOIN familyMembers ON groupMembers.id = familyMembers.groupMemberRelative WHERE groupMembers.name LIKE '%Calhoun%' GROUP BY groupMembers.name, groupMembers.age ", con=cnx) 

,name,age,AVG(familyMembers.age)
0,Anna Grace Calhoun,19,26.4


In [425]:
#Tohti family
pd.read_sql("SELECT groupMembers.name, groupMembers.age, AVG(familyMembers.age) FROM groupMembers JOIN familyMembers ON groupMembers.id = familyMembers.groupMemberRelative WHERE groupMembers.name LIKE '%Tohti%' GROUP BY groupMembers.name, groupMembers.age ", con=cnx)

,name,age,AVG(familyMembers.age)
0,Daniel Tohti,19,37.0


In [426]:
# Siamon family
pd.read_sql("SELECT groupMembers.name, groupMembers.age, AVG(familyMembers.age) FROM groupMembers JOIN familyMembers ON groupMembers.id = familyMembers.groupMemberRelative WHERE groupMembers.name LIKE '%Siamon%' GROUP BY groupMembers.name, groupMembers.age ", con=cnx)

,name,age,AVG(familyMembers.age)
0,David Siamon,7,42.6667


In [427]:
# Nguyen family
pd.read_sql("SELECT groupMembers.name, groupMembers.age, AVG(familyMembers.age) FROM groupMembers JOIN familyMembers ON groupMembers.id = familyMembers.groupMemberRelative WHERE groupMembers.name LIKE '%Nguyen%' GROUP BY groupMembers.name, groupMembers.age ", con=cnx)

,name,age,AVG(familyMembers.age)
0,Dare Nguyen,24,31.6


In [428]:
# Devito family
pd.read_sql("SELECT groupMembers.name, groupMembers.age, AVG(familyMembers.age) FROM groupMembers JOIN familyMembers ON groupMembers.id = familyMembers.groupMemberRelative WHERE groupMembers.name LIKE '%Devito%' GROUP BY groupMembers.name, groupMembers.age ", con=cnx)

,name,age,AVG(familyMembers.age)
0,Danny Devito,76,35.6667
